# Reduction in operations for union-of-stars using sparsification and decomposition

Corresponds to Table 2 in the paper

This notebook plots the reduction in the number of HIsing pulses, total number of operations (HIsing pulses and bit flips), and total time of HIsing pulses using sparse union-of-stars (i.e., using sparsification and decomposition techniques) as compared to classical union-of-stars algorithm for certain graphs in the MQLib library.

The data for the plots is stored in folder `data/MQLib_experiment_results` and was generated using the script `mqlib-instances.ipynb` in the `notebooks` folder.

In [ ]:
import pandas as pd
import os
import numpy as np

# Load the data

In [2]:
files = os.listdir('../data/MQLib_experiment_results/')
files = [f for f in files if f.endswith('.csv')]

df = pd.read_csv('../data/MQLib_experiment_results/' + files[0])
original_pulses = df[df['type'] == 'Original']['number_of_pulses'].values[0]
original_length = df[df['type'] == 'Original']['length_of_pulses'].values[0]
original_operations = (
        df[df['type'] == 'Original']['number_of_pulses'].values[0] +
        df[df['type'] == 'Original']['number_of_bit_flips'].values[0]
)
df['Frac pulses'] = df['number_of_pulses'] / original_pulses
df['Frac length'] = df['length_of_pulses'] / original_length
df['Frac operations'] = (df['number_of_pulses'] + df['number_of_bit_flips'])/ original_operations

for t in range(1, len(files)):
    df_instance = pd.read_csv('../data/MQLib_experiment_results/' + files[t])
    original_pulses = df_instance[df_instance['type'] == 'Original']['number_of_pulses'].values[0]
    original_length = df_instance[df_instance['type'] == 'Original']['length_of_pulses'].values[0]
    original_operations = (
            df_instance[df_instance['type'] == 'Original']['number_of_pulses'].values[0] +
            df_instance[df_instance['type'] == 'Original']['number_of_bit_flips'].values[0]
    )
    df_instance['Frac pulses'] = df_instance['number_of_pulses'] / original_pulses
    df_instance['Frac length'] = df_instance['length_of_pulses'] / original_length
    df_instance['Frac operations'] = (df_instance['number_of_pulses'] + df_instance['number_of_bit_flips'] )/ original_operations

    df = pd.concat([df, df_instance], ignore_index=True)


# Filter by number of vertices
df = df[(df['number_of_vertices'] >= 50) & (df['number_of_vertices'] < 200)]
# Filter by number of edges
df = df[(df['number_of_edges'] >= 2 * df['number_of_vertices']) & (df['number_of_edges'] < 2000)]

df = df[df['type'] != 'Original']
print(len(df['instance_name'].unique()))

194


# Group by sparsification and decomposition parameters

In [4]:
df

,instance_name,number_of_vertices,number_of_edges,type,epsilon_decomposition,decomposition_type,number_of_pulses,length_of_pulses,number_of_bit_flips,q_sparsification,seed_sparsification,epsilon_sparsification,frac_edges_sparsification,max_cut_approximation,Frac pulses,Frac length,Frac operations
77,g001649,160,812,Sparse,0.00,none,2437,88978.000000,3248,0.2,0.0,NaN,NaN,0.991744,1.000000,1.000000,1.000000
78,g001649,160,812,Sparse + Decomposed,0.10,exponential,421,84310.860740,570,0.2,0.0,NaN,NaN,0.771421,0.172753,0.947547,0.174318
79,g001649,160,812,Sparse + Decomposed,0.25,exponential,409,80468.945489,568,0.2,0.0,NaN,NaN,0.744889,0.167829,0.904369,0.171856
80,g001649,160,812,Sparse + Decomposed,0.50,exponential,388,70441.845199,550,0.2,0.0,NaN,NaN,0.810590,0.159212,0.791677,0.164996
81,g001649,160,812,Sparse + Decomposed,0.75,exponential,349,60216.147476,528,0.2,0.0,NaN,NaN,0.763630,0.143209,0.676753,0.154266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15738,g001863,160,812,Sparse + Decomposed,0.50,binary,4966,52189.148548,12464,2.0,0.0,NaN,NaN,0.929393,2.037751,0.417727,3.065963
15739,g001863,160,812,Sparse + Decomposed,0.75,binary,5050,54368.599621,12742,2.0,0.0,NaN,NaN,0.944324,2.072220,0.435172,3.129639
15740,g001863,160,812,Sparse + Decomposed,1.00,binary,4903,54930.278287,12112,2.0,0.0,NaN,NaN,0.922751,2.011900,0.439667,2.992964
15741,g001863,160,812,Sparse + Decomposed,2.00,binary,4540,54932.392532,11482,2.0,0.0,NaN,NaN,0.929888,1.862946,0.439684,2.818294


In [6]:
df_exp = df[df['decomposition_type'] == 'exponential']

columns = ['q_sparsification', 'epsilon_decomposition', 'Frac pulses', 'Frac length', 'Frac operations', 'max_cut_approximation']

df_parameters = df_exp.groupby(['q_sparsification', 'epsilon_decomposition']).agg({
    'Frac pulses': 'mean',
    'Frac length': 'mean',
    'Frac operations': 'mean',
    'max_cut_approximation': 'mean'
}).reset_index()

df_parameters = df_parameters[df_parameters['max_cut_approximation'] > 0.90]

In [8]:
df_parameters # This is Table 2 in the paper

,q_sparsification,epsilon_decomposition,Frac pulses,Frac length,Frac operations,max_cut_approximation
18,0.8,1.00,0.352415,0.569292,0.404287,0.900745
21,1.0,0.10,0.534672,0.870196,0.557148,0.914983
22,1.0,0.25,0.487506,0.779692,0.522866,0.917424
23,1.0,0.50,0.442998,0.680700,0.490973,0.913755
24,1.0,0.75,0.418665,0.614083,0.473762,0.916113
25,1.0,1.00,0.400020,0.566553,0.460231,0.915530
26,1.0,2.00,0.350625,0.438990,0.424993,0.912522
27,1.0,5.00,0.300766,0.298604,0.389230,0.912193
28,2.0,0.10,0.733430,0.873219,0.762057,0.948814
29,2.0,0.25,0.668303,0.773811,0.715505,0.949362
